In [1]:

import os
os.environ['OPENAI_API_KEY'] = ''
current_working_directory = '/drive/MyDrive/Colab Notebooks/<project name>/'  # Replace with your project name

import os
def check_if_colab_environment():
    try:
        import google.colab
        return True
    except ImportError:
        return False


if check_if_colab_environment() is True:
    from google.colab import drive
    drive.mount('/drive')


if current_working_directory.endswith('/'):
    current_working_directory = current_working_directory[:-1]

os.chdir(current_working_directory)


Mounted at /drive


In [2]:
# environment installation
!pip install chromadb
!pip install pysqlite3-binary
!pip install sqlite3 >= 3.35.0
!pip install langchain langchain_chroma langchain_community langchainhub langchain-openai
!pip install jq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3

In [14]:
from langchain_community.document_loaders import JSONLoader

#
push_content_loader = JSONLoader(
    file_path='./data/json/makeup.json',
    jq_schema='.[].messages[].push_content',
    text_content=False)

push_content = push_content_loader.load()  #Comment data of the post



loader = JSONLoader(
    file_path='./data/json/makeup.json',
    jq_schema='.[].content',
    text_content=False)

docs = loader.load() #Article data



In [ ]:
#Do data cleaning according to the analysis to be done.
import re

def remove_links(paragraph: str) -> str:
    # Regular expression to match URLs
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Substitute the URLs with an empty string
    cleaned_paragraph = re.sub(url_pattern, '', paragraph)

    return cleaned_paragraph

for doc in docs:
    doc.page_content = remove_links(doc.page_content)





In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

101662

In [16]:
len(all_splits[0].page_content)

818

In [43]:
all_splits[3].page_content

'廢話很大的網誌版  我一直覺得Burberry就是代表英國的名牌 既然包包大衣買不起 那化妝品總買得起了 現在Burberry beauty全球的櫃點還不多 查了資料發現倫敦的Harrods有 就決定去買了~  Harrods的網站在這裡 住在離倫敦比較遠的好像可以線上訂 不過我想試妝就直接殺去了 出發前我就已經在網路上看好要買腮紅了 到了專櫃後我跟小姐說我要試腮紅 Burberry的色號不多 我想要看粉色系的 就跟小姐說要試02和03 最後選了03，因為02有點暗，03比較年輕 小姐問我要不要試試別的 我就試試看粉底 她很細心的幫我在靠近下巴的臉上試了三種顏色 最後說 噢~這個太橘了~然後幫我挑了一個最合的色號 上了全臉以後 不知道是小姐服務好 還是怎樣的 妝效也還不錯 然後我就鬼迷心竅的把本來沒要買的粉底液帶回家了= = 小姐還給了我試用品 粉底液跟口紅  然後用精美的紙袋幫我裝起來 還打上緞帶 有夠高級的 我小心翼翼的把它捧回家了~  因為當初跑去試妝是很趕 等我上了車以後才想到我根本沒有拍到別的產品阿 甚至連口紅都沒看到沒摸到= = 好可惜 下次再去 廢話很多的分隔線 下面才是心得 Burberry粉底我買的是04號 我沒有注意其他顏色的色系 就是直接請小姐幫我試妝就買了     整個包裝很高貴 之前youtube的影片說它很重 其實我覺得還好 畢竟粉底液本來就比較不會帶出門   (照片都是黃光下照的>< 因為我房間都是黃色的燈，沒有白的) 粉底液擠出來有點像霜狀 但是擦起來沒有看起來的厚重 算是中等 不過我想在英國應該剛剛好 不會太乾太油 回台灣對油肌妹妹來說可能就會太滋潤了 遮瑕度我覺得算是不錯 當然黑眼圈跟大痘疤可能就要出動遮瑕膏了 粉質很細 不太會卡粉 04色號很貼我膚色 應該是膚色中偏白的女孩使用差不多 基本上我還算蠻滿意的 若真要說缺點 就是味道真的是很阿嬤 而且擦在臉上味道還久久不散= = 來講一下腮紅好了   腮紅很精緻還有絨布套 在側邊有寫上Burberry的字樣 是用磁鐵吸住上下的 感覺像是精品勝過於化妝品  上面壓有漂亮的菱格紋 不算壓很淺 應該要用好幾次才會平掉 我買的03是玫瑰色加上很細緻幾乎看不到的亮粉  其實Burberry腮紅顏色真的不多 也都是走高雅實用路線 對於有很多腮紅的人來說 顏色真的不太特別 不過粉很細 也還算持久

In [17]:
all_splits[10].metadata

{'source': '/drive/MyDrive/Colab Notebooks/好玩rag_makeup/data/json/makeup.json',
 'seq_num': 6,
 'start_index': 2409}

In [20]:
splits_test  = all_splits[:100]

In [42]:
splits_test[0]

Document(metadata={'source': '/drive/MyDrive/Colab Notebooks/好玩rag_makeup/data/json/makeup.json', 'seq_num': 1, 'start_index': 0}, page_content='最近大家的文章都越寫越好覺得欣慰又感動 你們真的好棒好棒Q__Q (感動) 所以這篇公告文章遲遲沒有發出 統整有一些版友的來信建議也讓大家參考一下 1.連結管理 因為blog的興盛很多產品心得和妝容分享都會帶上blog 大致上的blog除非要看圖片我都會優先看bbs文字版 管理上對錯字和注音的挑剔只限於bbs版本 因此無論blog裡面寫的再精采只提供連結會當作一行文(or不到20字)處理 也希望不要只提供連結版面文章雖超過20字但都跟標題無關 這類文章會開始打上s 2.連結請尊重原作者 要發他人的連結無論是要推崇或是和自己文章佐證 請先洽當事者並在文章開端先說明已經經過原作者的同意 沒有看到有事先詢問過的影子我自己解讀沒有詢問過 立即砍文尊重原作者 3.產品文管理 發表產品單品心得或是比較文的時候會希望版友多多補充基本資料 價錢/購買地/容量/產品大小等等 這不是強制要求但可以讓正在參考心得的版友多些資料當作參考依據 產品大小尤其在粉餅心得發表上更顯重要 粉蕊尺寸建議盡量寫出來對於想要用新粉蕊舊粉盒的版友就多了很多選擇 4.對著作權和創意的尊重 不支持版友對於雜誌的全面翻拍照一兩拿來仿妝容無妨 全面大量出現書本雜誌上無論文字或是圖片展現的文章都會大刀一揮 如從別人的文章/妝容得到靈感可以先在自己文章說明一下 能激發別人的靈感原作者也會很開心^^ 著作權法可以參考台灣的經濟部智慧財產局的法案  也可以撥打智慧局提供的服務電話(02)23767137或(02)23767182諮詢 裡面有提到的法案重申著作權的觀念 (擷取內文) 著作權四不一沒有的觀念，就是不要在網路上隨便免費下載音樂、影片、軟體； 不要將他人的文章、音樂、影片、軟體隨便轉寄；不要將他人的文章、照片隨便張貼或 移花接木；不要將他人的流行歌曲或音樂拿來在部落格使用；沒有合法授權的著作不要 在網路上販賣。 轉出版面文章請先通知原作者 版主cocolia')

In [50]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
vectorstore = Chroma.from_documents(documents=splits_test, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": })
retrieved_docs = retriever.invoke("Burberry 化妝品 回饋")

len(retrieved_docs)

15

In [46]:
print(retrieved_docs[1].page_content)

廢話很大的網誌版  我一直覺得Burberry就是代表英國的名牌 既然包包大衣買不起 那化妝品總買得起了 現在Burberry beauty全球的櫃點還不多 查了資料發現倫敦的Harrods有 就決定去買了~  Harrods的網站在這裡 住在離倫敦比較遠的好像可以線上訂 不過我想試妝就直接殺去了 出發前我就已經在網路上看好要買腮紅了 到了專櫃後我跟小姐說我要試腮紅 Burberry的色號不多 我想要看粉色系的 就跟小姐說要試02和03 最後選了03，因為02有點暗，03比較年輕 小姐問我要不要試試別的 我就試試看粉底 她很細心的幫我在靠近下巴的臉上試了三種顏色 最後說 噢~這個太橘了~然後幫我挑了一個最合的色號 上了全臉以後 不知道是小姐服務好 還是怎樣的 妝效也還不錯 然後我就鬼迷心竅的把本來沒要買的粉底液帶回家了= = 小姐還給了我試用品 粉底液跟口紅  然後用精美的紙袋幫我裝起來 還打上緞帶 有夠高級的 我小心翼翼的把它捧回家了~  因為當初跑去試妝是很趕 等我上了車以後才想到我根本沒有拍到別的產品阿 甚至連口紅都沒看到沒摸到= = 好可惜 下次再去 廢話很多的分隔線 下面才是心得 Burberry粉底我買的是04號 我沒有注意其他顏色的色系 就是直接請小姐幫我試妝就買了     整個包裝很高貴 之前youtube的影片說它很重 其實我覺得還好 畢竟粉底液本來就比較不會帶出門   (照片都是黃光下照的>< 因為我房間都是黃色的燈，沒有白的) 粉底液擠出來有點像霜狀 但是擦起來沒有看起來的厚重 算是中等 不過我想在英國應該剛剛好 不會太乾太油 回台灣對油肌妹妹來說可能就會太滋潤了 遮瑕度我覺得算是不錯 當然黑眼圈跟大痘疤可能就要出動遮瑕膏了 粉質很細 不太會卡粉 04色號很貼我膚色 應該是膚色中偏白的女孩使用差不多 基本上我還算蠻滿意的 若真要說缺點 就是味道真的是很阿嬤 而且擦在臉上味道還久久不散= = 來講一下腮紅好了   腮紅很精緻還有絨布套 在側邊有寫上Burberry的字樣 是用磁鐵吸住上下的 感覺像是精品勝過於化妝品  上面壓有漂亮的菱格紋 不算壓很淺 應該要用好幾次才會平掉 我買的03是玫瑰色加上很細緻幾乎看不到的亮粉  其實Burberry腮紅顏色真的不多 也都是走高雅實用路線 對於有很多腮紅的人來說 顏色真的不太特別 不過粉很細 也還算持久


In [52]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("Burberry 化妝品 關鍵字 "):
    print(chunk, end="", flush=True)

Burberry的化妝品在Harrods有櫃點，可以前往購買。試妝後購買的粉底液和腮紅都讓顧客感到滿意。外帶的唇膏顏色也適合秋天的氛圍。